## Setup

In [1]:
%cd /content/
!git clone https://github.com/DengPingFan/SINet.git
%cd SINet/
!pip install opencv-python
!pip install gdown

%mkdir Dataset
%cd Dataset
!gdown 1D9bf1KeeCJsxxri6d2qAC7z6O1X_fxpt
!gdown 1QEGnP9O7HbN_2tH999O3HRIsErIVYalx
!unzip TrainDataset.zip
!unzip TestDataset.zip
!mv ./TestDataset/COD10K/Imgs ./TestDataset/COD10K/Image
!mv ./TrainDataset/Imgs ./TrainDataset/Image


%cd ..

Streaming output truncated to the last 5000 lines.
  inflating: TestDataset/COD10K/Imgs/COD10K-CAM-1-Aquatic-13-Pipefish-591.jpg  
  inflating: TestDataset/COD10K/Imgs/COD10K-CAM-1-Aquatic-13-Pipefish-590.jpg  
  inflating: TestDataset/COD10K/Imgs/COD10K-CAM-1-Aquatic-13-Pipefish-588.jpg  
  inflating: TestDataset/COD10K/Imgs/COD10K-CAM-1-Aquatic-13-Pipefish-585.jpg  
  inflating: TestDataset/COD10K/Imgs/COD10K-CAM-1-Aquatic-13-Pipefish-583.jpg  
  inflating: TestDataset/COD10K/Imgs/COD10K-CAM-1-Aquatic-13-Pipefish-582.jpg  
  inflating: TestDataset/COD10K/Imgs/COD10K-CAM-1-Aquatic-13-Pipefish-580.jpg  
  inflating: TestDataset/COD10K/Imgs/COD10K-CAM-1-Aquatic-13-Pipefish-579.jpg  
  inflating: TestDataset/COD10K/Imgs/COD10K-CAM-1-Aquatic-13-Pipefish-577.jpg  
  inflating: TestDataset/COD10K/Imgs/COD10K-CAM-1-Aquatic-13-Pipefish-576.jpg  
  inflating: TestDataset/COD10K/Imgs/COD10K-CAM-1-Aquatic-13-Pipefish-573.jpg  
  inflating: TestDataset/COD10K/Imgs/COD10K-CAM-1-Aquatic-13-Pipefish

In [2]:
!git clone https://github.com/NVIDIA/apex
%cd apex
!python3 setup.py install
!pip install packaging
%cd ..

Cloning into 'apex'...
remote: Enumerating objects: 11638, done.
remote: Counting objects: 100% (3731/3731), done.
remote: Compressing objects: 100% (579/579), done.
remote: Total 11638 (delta 3371), reused 3280 (delta 3149), pack-reused 7907
Receiving objects: 100% (11638/11638), 15.47 MiB | 20.57 MiB/s, done.
Resolving deltas: 100% (8173/8173), done.
/content/SINet/apex


torch.__version__  = 2.2.1+cu121


running install
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer, pypa/build or
        other standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.

In [6]:
!gdown 1iCkHXU55Or4gEtSqesijea6A5r8lVEYv

Downloading...
From (original): https://drive.google.com/uc?id=1iCkHXU55Or4gEtSqesijea6A5r8lVEYv
From (redirected): https://drive.google.com/uc?id=1iCkHXU55Or4gEtSqesijea6A5r8lVEYv&confirm=t&uuid=ff338466-3e77-4018-8140-fbbaf1be3294
To: /content/SINet/SINet_40.pth
100% 196M/196M [00:00<00:00, 231MB/s]


## Eval code

In [7]:
import cv2
import os
import tqdm

class SemSegMetrics():
  def __init__(self):
    self.metrics = {
        "miou": 0,
        "pixel_acc": 0,
        "dice": 0,
    }
    self.count = 0

  def update(self, pred, gt):
    self.count += 1
    self.metrics["pixel_acc"] += (pred == gt).sum()/(gt.shape[0] * gt.shape[1])
    self.metrics["dice"] += 2 * (pred * gt).sum()/(pred.sum() + gt.sum())
    self.metrics["miou"] += (pred * gt).sum()/((pred + gt)>0).sum()

  def __str__(self):
    met = {}
    for k in self.metrics:
      met[k] = self.metrics[k]/ self.count

    s = " | ".join(["{}: {:.2f}".format(k, 100*met[k]) for k in met])
    return s



def eval_metrics(result_folder, gt_folder):
  metrics = SemSegMetrics()
  for fname in tqdm.tqdm(os.listdir(result_folder)):
    pred = cv2.imread(os.path.join(result_folder, fname), 0)
    gt   = cv2.imread(os.path.join(gt_folder, fname), 0)
    # print(pred.shape, pred.max(), pred.min())
    # print(gt.shape, gt.max(), gt.min())
    # break
    metrics.update(pred>0, gt>0)

  print()
  print(metrics)




## Run Train and test

In [ ]:
!python3 MyTrain.py --gpu 0

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
[INFO] initialize weights from resnet50
------------------------------ SINet_ResNet50(
  (resnet): ResNet_2Branch(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, 

In [ ]:
!cp /content/SINet/Snapshot/2020-CVPR-SINet/SINet_40.pth /content/drive/MyDrive/CourseWork/2024\ Spring/cs670/SINet/
!python3 MyTest.py --model_path /content/SINet/Snapshot/2020-CVPR-SINet/SINet_40.pth

In [5]:
!python3 MyTest.py --model_path /content/SINet/SINet_40.pth

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100% 97.8M/97.8M [00:01<00:00, 61.5MB/s]
[INFO] initialize weights from resnet50
Traceback (most recent call last):
  File "/content/SINet/MyTest.py", line 21, in <module>
    model.load_state_dict(torch.load(opt.model_path))
  File "/usr/local

In [ ]:
eval_metrics(
    "/content/SINet/Result/2020-CVPR-SINet-New/COD10K",
    "/content/SINet/Dataset/TestDataset/COD10K/GT"
)

100%|██████████| 2026/2026 [00:45<00:00, 44.82it/s]


miou: 49.00 | pixel_acc: 82.41 | dice: 61.63
